# Train Models

In [1]:
import warnings
import pickle
import gensim

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

from sklearn.manifold import TSNE
import pandas as pd

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [39]:
%%time

# initiate the model and perform the first epoch of training
skill2vec = Word2Vec(unigram_sentences, size=50, window=5,
                    min_count=20, sg=0, hs=1, negative=0, workers=8, iter=15, compute_loss = True)


skill2vec.save(word2vec_filepath)
print("Done generating model.")

# perform 10 epochs of training
skill2vec.train(unigram_sentences,total_examples=skill2vec.corpus_count, epochs=skill2vec.iter)
skill2vec.save(word2vec_filepath)

# Start 2:45

/usr/local/lib/python3.6/site-packages/gensim/utils.py:598: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  if hasattr(self, attrib):
/usr/local/lib/python3.6/site-packages/gensim/utils.py:599: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  asides[attrib] = getattr(self, attrib)
/usr/local/lib/python3.6/site-packages/gensim/utils.py:600: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  delattr(self, attrib)
/usr/local/lib/python3.6/site-packages/gensim/utils.py:555: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(obj, attrib, val)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will 

Done generating model.


/usr/local/lib/python3.6/site-packages/gensim/utils.py:598: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  if hasattr(self, attrib):
/usr/local/lib/python3.6/site-packages/gensim/utils.py:599: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  asides[attrib] = getattr(self, attrib)
/usr/local/lib/python3.6/site-packages/gensim/utils.py:600: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  delattr(self, attrib)


CPU times: user 1h 47min 30s, sys: 54 s, total: 1h 48min 24s
Wall time: 52min 1s


/usr/local/lib/python3.6/site-packages/gensim/utils.py:555: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(obj, attrib, val)


In [ ]:
unigram_sentences = LineSentence('../data/all_jobs_bigrams.txt')
word2vec_filepath = '../models/word2vec_hs1_neg0_window_15_size300'

In [7]:
# Training loss
0.0 == skill2vec.get_latest_training_loss()

True

In [3]:
# load a finished model from disk
skill2vec = Word2Vec.load(word2vec_filepath)
skill2vec.init_sims()
print(u'Model loaded with {} training epochs.'.format(skill2vec.train_count))

Model loaded with 2 training epochs.


/usr/local/lib/python3.6/site-packages/gensim/utils.py:491: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)


In [4]:
print(u'{:,} terms in the skill2vec vocabulary.'.format(len(skill2vec.wv.vocab)))

55,001 terms in the skill2vec vocabulary.


In [5]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in skill2vec.wv.most_similar(positive=[token], topn=topn):

        print(u'{:30} {}'.format(word, round(similarity, 3)))

In [7]:
get_related_terms(u'postgresql')

postgre                        0.85
mysql                          0.742
mssql                          0.687
mariadb                        0.686
mongodb                        0.681
postgr                         0.68
sqlite                         0.66
pgsql                          0.653
mongo                          0.642
oracle                         0.635
database                       0.63
psql                           0.62
rdbms                          0.619
sqlite3                        0.581
dbms                           0.576
mysql_server                   0.574
sqlserver                      0.574
postgress                      0.569
pgadmin                        0.563
db2                            0.558


In [8]:
get_related_terms(u'neural_net') 

neural_network                 0.827
neural_networks                0.805
decision_tree                  0.8
support_vector                 0.78
bayesian                       0.763
survival_analysis              0.762
svms                           0.76
svm                            0.756
boosting                       0.717
baye                           0.717
ensemble                       0.708
ensembl                        0.706
bagging                        0.704
association_rule               0.699
cluster_analysis               0.694
clustering                     0.693
naive_baye                     0.692
chaid                          0.691
knn                            0.69
markov                         0.689


In [11]:
get_related_terms(u'certified_scrummaster')

cspo                           0.689
certified_scrum                0.671
acp                            0.646
pmp                            0.623
prince2                        0.618
pgmp                           0.603
pmi                            0.601
capm                           0.586
cbap                           0.56
csm                            0.524
certification                  0.518
certification(s                0.477
mcsd                           0.47
pmbok                          0.465
certify                        0.465
dawia                          0.45
itil                           0.44
greenhopper                    0.435
confluence                     0.43
csslp                          0.423


In [12]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = skill2vec.wv.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [16]:
word_algebra(add=[u'nosql', u'postgresql'], subtract=[u'mongodb'])

rdbms


In [17]:
word_algebra(add=[u'python', u'sql'])

postgresql


In [18]:
word_algebra(add=[u'oop', u'javascript'], 
             subtract=[u'php'])

object_oriented


In [22]:
word_algebra(add=[u'product', u'development'])

production


In [26]:
word_algebra(add=[u'sql', u'git'])

checkout


# Visualize

In [27]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in skill2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda item: -item[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the skill2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(skill2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  


In [28]:
tsne_input = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [29]:
tsne_input.head()

0         1         2         3         4         5         6    \
-PRON-  0.096944  0.087000  0.003583 -0.001504  0.111694 -0.096787 -0.011931   
gt      0.037769  0.018485 -0.005161 -0.009153 -0.049085 -0.093955  0.039386   
lt      0.042331  0.023457 -0.036514 -0.013101 -0.047667 -0.065932  0.078685   
use     0.033183 -0.015958  0.024219 -0.001740  0.088988 -0.050933  0.019642   
work    0.010562  0.065636 -0.004473 -0.072205  0.072555 -0.082940 -0.065306   

             7         8         9      ...          290       291       292  \
-PRON-  0.088636  0.051716 -0.038965    ...    -0.053500 -0.007269  0.047801   
gt      0.001749  0.014394 -0.039650    ...    -0.021234  0.003288  0.140201   
lt      0.022556  0.013112 -0.027004    ...     0.011652 -0.023489  0.129856   
use     0.079626  0.042988  0.018514    ...    -0.084227 -0.006201  0.048437   
work    0.032511  0.113921 -0.036668    ...    -0.083731  0.003530  0.065099   

             293       294       295       296       297       298       299  
-PRON-  0.031715  0.053776  0.070925  0.107324  0.016900 -0.019526  0.039425  
gt     -0.060737  0.015184  0.029666 -0.032321  0.026446 -0.009485  0.099500  
lt     -0.032377  0.008627  0.027702 -0.047348  0.031673 -0.016822  0.107282  
use     0.056621  0.101004  0.049623  0.111949  0.062625  0.061967  0.067629  
work   -0.039295  0.014586  0.026540  0.060570  0.087757 -0.005825  0.021195  

[5 rows x 300 columns]

In [48]:
tsne_filepath = u'models/tsne_model'
tsne_vectors_filepath = u'models/tsne_vectors.npy'

In [50]:
%%time

tsne = TSNE()
tsne_vectors = tsne.fit_transform(tsne_input.values)

with open(tsne_filepath, 'wb') as f:
    pickle.dump(tsne, f)

pd.np.save(tsne_vectors_filepath, tsne_vectors)

In [51]:
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

In [52]:
tsne_vectors[u'word'] = tsne_vectors.index

In [53]:
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()
# output_file("models/tsne_bokeh.html")

Loading BokehJS ...

In [54]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 1000,
                   plot_height = 1000,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'green', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

# Assess Model on Original Word2vec Analogy Task

In [55]:
def model_accuracy(model, questions):

    accuracy = model.wv.accuracy(questions)
    
    sum_corr = len(accuracy[-1]['correct'])
    sum_incorr = len(accuracy[-1]['incorrect'])
    total = sum_corr + sum_incorr
    percent = lambda a: a / total * 100
    
    print('Total tasks: {}, Correct: {:.2f}%, Incorrect: {:.2f}%'.format(total, percent(sum_corr), percent(sum_incorr)))

In [56]:
model_accuracy(skill2vec, 'question-words.txt')

Total tasks: 3212, Correct: 28.08%, Incorrect: 71.92%


In [57]:
# skill2vec.wv.accuracy('question-words.txt')

## Clustering (for use with Tensorboard)

In [35]:
X = skill2vec[skill2vec.wv.vocab]
words = list(skill2vec.wv.vocab)

/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [45]:
%%time
from sklearn import cluster
from sklearn import metrics
kmeans = cluster.KMeans(n_clusters=100)
kmeans.fit(X)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_
print(labels[:10])
print(words[:10])

[75 70 96 48 48 64 48 48 48 64]
['factory', 'automation', 'software_engineer', 'to', 'support', 'semiconductor', 'development', 'in', 'a', 'semiconductor_equipment']
CPU times: user 4min 35s, sys: 4min 28s, total: 9min 4s
Wall time: 3min 53s


In [46]:
sklearn_clust_dict = {}

for i, word in enumerate(words):  
    sklearn_clust_dict[word] = str(labels[i])

In [47]:
# Test some terms
sample_words = ['factory', 'automation', 'software_engineer', 'to', 
                'support', 'semiconductor', 'development', 'in', 'a', 
                'semiconductor_equipment']

for word in sample_words:
    print(sklearn_clust_dict[word])

75
70
96
48
48
64
48
48
48
64


In [48]:
print(len(sklearn_clust_dict.keys()))

91590


## Put the clusters into a metadata file, for ingestion by Tensorboard

In [51]:
# We'll overwrite this file
!rm ../projector/v2/w2x_metadata.tsv

In [52]:
%%time
# Create updated tsv file for uploading to Tensorboard
with open('../projector/w2x_metadata.tsv', 'r') as infile:
    with open('../projector/v2/w2x_metadata.tsv', 'w+') as outfile:
        outfile.write('label\tcluster\n')
        for line in infile:
            word = line.replace('\n','')
            clust = sklearn_clust_dict[word]
            outfile.write(word)
            outfile.write('\t')
            outfile.write(clust)
            outfile.write('\n')

CPU times: user 357 ms, sys: 16.6 ms, total: 373 ms
Wall time: 362 ms


In [60]:
hard_skill_clusters = []

hard_skill_examples = ['postgresql', 'sql', 'python', 'java', 'r', 
                'ubuntu', 'linux', 'perl', 'c++', 'oop', 'git', 'jira',
               'cloud_compute', 'elasticsearch', 'cassandra', 'kafka', 
               'photoshop', 'invision', 'js', 'excel', 'unit_test', 'regression_test',
               'automate']
print('#'*10,"Hard skill words:", '#'*10)
for word in hard_skill_examples:
    print(word,":",sklearn_clust_dict[word])
    hard_skill_clusters.append(sklearn_clust_dict[word])
    
bad_words = ['program', 'job', 'sale', 'obvious']
print('\n','#'*10,"Non-skill words:", '#'*10)
for word in bad_words:
    print(word,":",sklearn_clust_dict[word])
    
soft_skills = ['interpersonal', 'organizational', 'listening', 'communication_skill', 'adaptable']
print('\n','#'*10, 'Soft skill words:', '#'*10)
for word in soft_skills:
    print(word,":",sklearn_clust_dict[word])

########## Hard skill words: ##########
postgresql : 52
sql : 52
python : 99
java : 100
r : 66
ubuntu : 75
linux : 11
perl : 50
c++ : 50
oop : 50
git : 93
jira : 51
cloud_compute : 68
elasticsearch : 68
cassandra : 68
kafka : 68
photoshop : 69
invision : 69
js : 179
excel : 196
unit_test : 32
regression_test : 32
automate : 47

 ########## Non-skill words: ##########
program : 21
job : 21
sale : 4
obvious : 135

 ########## Soft skill words: ##########
interpersonal : 191
organizational : 191
listening : 191
communication_skill : 191
adaptable : 146


In [61]:
unique_hard_skill_clusts = [int(item) for item in list(set(sorted(hard_skill_clusters)))]
print("Clusters containing hard skills:", unique_hard_skill_clusts)

Clusters containing hard skills: [51, 11, 32, 66, 93, 50, 52, 196, 68, 75, 99, 100, 47, 179, 69]


In [62]:
# This is the list of hard skills that we can use later.
all_skill_words = [k for k,v in sklearn_clust_dict.items() if int(v) in unique_hard_skill_clusts]

In [63]:
# Write the list to file for later use.
with open('../models_05apr/hard_skills_from_nn2.txt', 'w') as outfile:
    for item in sorted(all_skill_words):
        outfile.write(item)
        outfile.write('\n')

In [2]:
# !tail -100 ../models_data_lower/hard_skills_from_nn.txt